# Notes

In [1]:
## Point selection after nearest location isn't random, defaults to lowest value index

# Imports

In [2]:
##IMPORTS
import numpy as np
from numpy import *
import copy

# Take in or create input

In [3]:
## FOR THE BIG 600 CELL ONE
import csv

with open("sixhundredcell_3_corrected.csv") as file_name:
    inputMatrixFull = np.loadtxt(file_name, dtype = 'int', delimiter=",")
    
    
    
inputMatrix = inputMatrixFull[:]

    
## FOR SMALLER TESTING SOLIDS
#inputMatrix  = 



    
print(inputMatrix)


[[  1  29  43  49]
 [  1  29  36  43]
 [  1  25  26  30]
 ...
 [ 89 106 111 116]
 [100 106 116 120]
 [ 89 100 106 116]]


# Create master array for which faces have and haven't been covered

In [4]:
## Create Master Array of Uncovered Faces
## Will be removed from throughout process
masterUncoveredFaces = copy.deepcopy(inputMatrix)
#print(masterUncoveredFaces)

##Create Master Array for covered faces, starts empty, fills to be inputMatrix
rows, cols = (1, len(inputMatrix[0]))
masterCoveredFaces = [[-1 for i in range(cols)] for j in range(rows)]
#print(masterCoveredFaces)

# Find total number of vertices

In [5]:
## Find number of vertices
inputMatrixWidth= len(inputMatrix[0])
maxVert = 0
for x in inputMatrix:
    if x[inputMatrixWidth-1] > maxVert:
        maxVert = x[inputMatrixWidth-1]

print(maxVert)

120


# Creating the Dictionary of each face that touches each vertex

In [6]:
## Create list of faces touched by each vertex, as well as copy for strictly uncovered

## How to know how many faces each vertex will touch? **************************
    ## For platonic solid, equal for all, so find max faces for one, for all
    ## Just make it a dictionary, probably better
   

vertexDictOfTouchedFaces = {
    
}

for nextVertex in range(maxVert):
    vertexDictOfTouchedFaces[nextVertex+1] = []
    
## Fill in the dictionary
for currentFace in inputMatrix:
    for currentVertex in currentFace:
            vertexDictOfTouchedFaces[currentVertex].append(currentFace)
            
#print(vertexDictOfTouchedFaces)


##Create the dictionary for each uncovered face, will be removed from throughout the process
vertexDictOfTouchingUncoveredFaces = copy.deepcopy(vertexDictOfTouchedFaces)

#print(vertexDictOfTouchedFaces[8])

vertexDictOfTouchedFacesEnding = {}
for nextVertex in range(maxVert):
    vertexDictOfTouchedFacesEnding[nextVertex+1] = []
    
## Fill in the dictionary
for currentFace in inputMatrixFull:
    for currentVertex in currentFace:
            vertexDictOfTouchedFacesEnding[currentVertex].append(currentFace)
            
print(vertexDictOfTouchedFacesEnding)

{1: [array([ 1, 29, 43, 49]), array([ 1, 29, 36, 43]), array([ 1, 25, 26, 30]), array([ 1, 25, 26, 29]), array([ 1, 25, 36, 37]), array([ 1, 33, 36, 37]), array([ 1, 33, 40, 44]), array([ 1, 33, 40, 43]), array([ 1, 40, 49, 50]), array([ 1, 26, 49, 50]), array([ 1, 30, 44, 50]), array([ 1, 30, 37, 44]), array([ 1, 25, 29, 36]), array([ 1, 33, 36, 43]), array([ 1, 40, 43, 49]), array([ 1, 26, 29, 49]), array([ 1, 25, 30, 37]), array([ 1, 33, 37, 44]), array([ 1, 40, 44, 50]), array([ 1, 26, 30, 50])], 2: [array([ 2, 29, 56, 62]), array([ 2, 29, 36, 62]), array([ 2, 25, 27, 32]), array([ 2, 25, 27, 29]), array([ 2, 25, 36, 39]), array([ 2, 34, 36, 39]), array([ 2, 34, 53, 64]), array([ 2, 34, 53, 62]), array([ 2, 53, 56, 58]), array([ 2, 27, 56, 58]), array([ 2, 32, 58, 64]), array([ 2, 32, 39, 64]), array([ 2, 25, 29, 36]), array([ 2, 34, 36, 62]), array([ 2, 53, 56, 62]), array([ 2, 27, 29, 56]), array([ 2, 25, 32, 39]), array([ 2, 34, 39, 64]), array([ 2, 53, 58, 64]), array([ 2, 27, 

# Create Dictionary Of Which Vertices a Single Vertex shares edges with

In [7]:
##Create something that stores every vertex, and each other vertex it shares an edge with
    ## shares edge if it shares 2 faces
    ##Look in vertexDictOfTouchedFaces, keep count of each time each vertex appears. If > 1 and not the original, edge
##What to store in, dict doesn't seem to be working well, hard with arrays as values
    ## Dict for now
    
vertexDictOfSharedEdges = {}

for nextVertex in range(maxVert):
    vertexDictOfSharedEdges[nextVertex+1] = []

vertexDictOfVerticesOnEdge = {    
}

for nextVertex in range(maxVert):
    vertexDictOfVerticesOnEdge[nextVertex+1] = []

rows, cols = (maxVert, maxVert)
vertexSharedFacesCount = [[0 for i in range(cols)] for j in range(rows)]
#print(len(vertexSharedFacesCount)*len(vertexSharedFacesCount[0]))



for currentVertex in vertexDictOfTouchedFaces:
    
    #print(currentVertex, vertexDictOfTouchedFaces[currentVertex])
    for currentFace in vertexDictOfTouchedFaces[currentVertex]:
        for vertexOnFace in currentFace:
            vertexSharedFacesCount[currentVertex-1][vertexOnFace-1] += 1
            if vertexSharedFacesCount[currentVertex-1][vertexOnFace-1] > 1 and vertexOnFace not in vertexDictOfSharedEdges[currentVertex]:
                    if vertexOnFace != currentVertex:
                        vertexDictOfSharedEdges[currentVertex].append(vertexOnFace)
    
    ##PUT THIS INSIDE LOOP ABOVE, SO TO ACCESS THE VALUE OF vertexOnFace
    ## Also, use number of faces each vertex is on to eliminate vertices pairing wiht self
    ##for x in vertexSharedFacesCount[currentVertex-1]:
        ##if x > 1:
            ##print(x, currentVertex)
            


#print(vertexSharedFacesCount)
#print(vertexDictOfSharedEdges)

tempVertexDictOfSharedEdges = copy.deepcopy(vertexDictOfSharedEdges)
#print(tempVertexDictOfSharedEdges)

#Once this removes an element from the dictionary, it doesn't check the next value. use temp dictionary
for currentVertex in vertexDictOfSharedEdges:
    for connectedVertex in vertexDictOfSharedEdges[currentVertex]:
        #print(currentVertex, connectedVertex)
        if connectedVertex == currentVertex:
            tempVertexDictOfSharedEdges[currentVertex].remove(connectedVertex)
            #print("removed", connectedVertex)
            
#print(vertexDictOfSharedEdges)

vertexDictOfSharedEdges = copy.deepcopy(tempVertexDictOfSharedEdges)
    
print(vertexDictOfSharedEdges)  







{1: [29, 43, 25, 26, 36, 37, 33, 40, 49, 50, 30, 44], 2: [29, 62, 25, 27, 36, 39, 34, 53, 56, 58, 32, 64], 3: [43, 73, 40, 41, 49, 51, 47, 66, 68, 70, 46, 74], 4: [43, 62, 36, 38, 33, 41, 68, 69, 34, 60, 45, 63], 5: [29, 73, 26, 27, 49, 52, 47, 54, 56, 57, 31, 75], 6: [62, 73, 56, 59, 53, 60, 68, 71, 54, 66, 65, 76], 7: [86, 97, 80, 83, 77, 84, 92, 95, 78, 90, 89, 100], 8: [44, 86, 37, 38, 33, 42, 92, 93, 35, 84, 45, 87], 9: [30, 97, 26, 28, 80, 81, 78, 48, 50, 52, 31, 99], 10: [44, 97, 40, 42, 92, 94, 90, 48, 50, 51, 46, 98], 11: [30, 86, 25, 28, 37, 39, 35, 77, 80, 82, 32, 88], 12: [87, 113, 84, 85, 108, 111, 107, 91, 93, 95, 89, 116], 13: [45, 113, 41, 42, 69, 70, 67, 91, 93, 94, 46, 114], 14: [63, 87, 38, 39, 34, 61, 108, 109, 35, 85, 64, 88], 15: [63, 113, 60, 61, 108, 110, 107, 67, 69, 71, 65, 115], 16: [98, 114, 94, 95, 90, 96, 117, 120, 91, 112, 100, 116], 17: [74, 98, 51, 52, 47, 72, 117, 118, 48, 96, 75, 99], 18: [74, 114, 70, 71, 66, 72, 117, 119, 67, 112, 76, 115], 19: [99,

# Vertex Shared Edges Ending

In [8]:
##Create something that stores every vertex, and each other vertex it shares an edge with
    ## shares edge if it shares 2 faces
    ##Look in vertexDictOfTouchedFaces, keep count of each time each vertex appears. If > 1 and not the original, edge
##What to store in, dict doesn't seem to be working well, hard with arrays as values
    ## Dict for now
    
vertexDictOfSharedEdgesEnding = {}

for nextVertex in range(maxVert):
    vertexDictOfSharedEdgesEnding[nextVertex+1] = []

vertexDictOfVerticesOnEdgeEnding = {    
}

for nextVertex in range(maxVert):
    vertexDictOfVerticesOnEdgeEnding[nextVertex+1] = []

rows, cols = (maxVert, maxVert)
vertexSharedFacesCountEnding = [[0 for i in range(cols)] for j in range(rows)]
#print(len(vertexSharedFacesCount)*len(vertexSharedFacesCount[0]))



for currentVertex in vertexDictOfTouchedFacesEnding:
    
    #print(currentVertex, vertexDictOfTouchedFaces[currentVertex])
    for currentFace in vertexDictOfTouchedFacesEnding[currentVertex]:
        for vertexOnFace in currentFace:
            vertexSharedFacesCountEnding[currentVertex-1][vertexOnFace-1] += 1
            if vertexSharedFacesCountEnding[currentVertex-1][vertexOnFace-1] > 1 and vertexOnFace not in vertexDictOfSharedEdgesEnding[currentVertex]:
                    if vertexOnFace != currentVertex:
                        vertexDictOfSharedEdgesEnding[currentVertex].append(vertexOnFace)
    
    ##PUT THIS INSIDE LOOP ABOVE, SO TO ACCESS THE VALUE OF vertexOnFace
    ## Also, use number of faces each vertex is on to eliminate vertices pairing wiht self
    ##for x in vertexSharedFacesCount[currentVertex-1]:
        ##if x > 1:
            ##print(x, currentVertex)
            


#print(vertexSharedFacesCount)
#print(vertexDictOfSharedEdges)

tempVertexDictOfSharedEdgesEnding = copy.deepcopy(vertexDictOfSharedEdgesEnding)
#print(tempVertexDictOfSharedEdges)

#Once this removes an element from the dictionary, it doesn't check the next value. use temp dictionary
for currentVertex in vertexDictOfSharedEdgesEnding:
    for connectedVertex in vertexDictOfSharedEdgesEnding[currentVertex]:
        #print(currentVertex, connectedVertex)
        if connectedVertex == currentVertex:
            tempVertexDictOfSharedEdgesEnding[currentVertex].remove(connectedVertex)
            #print("removed", connectedVertex)
            
#print(vertexDictOfSharedEdges)

vertexDictOfSharedEdgesEnding = copy.deepcopy(tempVertexDictOfSharedEdgesEnding)
    
print(vertexDictOfSharedEdgesEnding)

{1: [29, 43, 25, 26, 36, 37, 33, 40, 49, 50, 30, 44], 2: [29, 62, 25, 27, 36, 39, 34, 53, 56, 58, 32, 64], 3: [43, 73, 40, 41, 49, 51, 47, 66, 68, 70, 46, 74], 4: [43, 62, 36, 38, 33, 41, 68, 69, 34, 60, 45, 63], 5: [29, 73, 26, 27, 49, 52, 47, 54, 56, 57, 31, 75], 6: [62, 73, 56, 59, 53, 60, 68, 71, 54, 66, 65, 76], 7: [86, 97, 80, 83, 77, 84, 92, 95, 78, 90, 89, 100], 8: [44, 86, 37, 38, 33, 42, 92, 93, 35, 84, 45, 87], 9: [30, 97, 26, 28, 80, 81, 78, 48, 50, 52, 31, 99], 10: [44, 97, 40, 42, 92, 94, 90, 48, 50, 51, 46, 98], 11: [30, 86, 25, 28, 37, 39, 35, 77, 80, 82, 32, 88], 12: [87, 113, 84, 85, 108, 111, 107, 91, 93, 95, 89, 116], 13: [45, 113, 41, 42, 69, 70, 67, 91, 93, 94, 46, 114], 14: [63, 87, 38, 39, 34, 61, 108, 109, 35, 85, 64, 88], 15: [63, 113, 60, 61, 108, 110, 107, 67, 69, 71, 65, 115], 16: [98, 114, 94, 95, 90, 96, 117, 120, 91, 112, 100, 116], 17: [74, 98, 51, 52, 47, 72, 117, 118, 48, 96, 75, 99], 18: [74, 114, 70, 71, 66, 72, 117, 119, 67, 112, 76, 115], 19: [99,

# Fill out matrix of vertex to vertex distances

In [9]:
rows, cols = (maxVert, maxVert)
vertexToVertexDistance = [[999 for i in range(cols)] for j in range(rows)]
#print(vertexToVertexDistance)

for i in range(0,maxVert):
    vertexToVertexDistance[i][i] = 0

#print( vertexToVertexDistance)

edgesAway = 1

while np.amax(vertexToVertexDistance) == 999:
    for vertex in vertexDictOfSharedEdges:
        for secondVertex in vertexDictOfSharedEdges:
            if vertexToVertexDistance[vertex-1][secondVertex-1] == edgesAway-1:
                #print(secondVertex, " is ", edgesAway, " away")
                for vertXAway in vertexDictOfSharedEdges[secondVertex]:
                    if vertexToVertexDistance[vertex-1][vertXAway-1] == 999:
                        vertexToVertexDistance[vertex-1][vertXAway-1] = edgesAway
    edgesAway += 1
#print(vertexToVertexDistance)
  
#print(vertexToVertexDistance)
#print(np.amax(vertexToVertexDistance))

# Tiebreaker: Calculate the distance from each maxUntouchedVertex to a covered face

In [10]:
## Create function for running calucations on each vertex in an input list
## Simply returns which of the input vertices is closest

## Premake the matrix of distances

## When called, search through covered faces
## Pull out every vertex in the covered faces sets

## For each maxUntouchedVertex, find distance each vertex in covered faces sets, take min
## Store min as distance from covered faces

## Find maxUntouchedVertex with smallest distance from covered faces

## Return that maxUntouchedVertex

def findNextVertex(listOfMaxedFaceVerts):
    
    ## Fill out all vert touching a covered face
    verticesTouchingCoveredFaces = []
    for coveredFace in masterCoveredFaces:
        for vertex in coveredFace:
            if vertex not in verticesTouchingCoveredFaces:
                #print( vertex, "isn't in verticesTouchingCoveredFaces")
                verticesTouchingCoveredFaces.append(vertex)
                #print("verticesTouchingCoveredFaces is now ", verticesTouchingCoveredFaces)
    dictOfMaxedVertDistanceToFace = {}            
    for vertex in listOfMaxedFaceVerts:
        dictOfMaxedVertDistanceToFace[vertex] = []
        closestDistance = 999999
        for vertOnFace in verticesTouchingCoveredFaces:
            if vertexToVertexDistance[vertex-1][vertOnFace -1] < closestDistance:
                closestDistance = vertexToVertexDistance[vertex-1][vertOnFace -1]
        #print(vertex, closestDistance)
        dictOfMaxedVertDistanceToFace[vertex] = closestDistance
            
        
    #print(dictOfMaxedVertDistanceToFace)

    ## Find maxUntouchedVertex with smallest distance from covered faces
    closestVert = 9999
    closestVertDist = 9999
    
    for vertex in dictOfMaxedVertDistanceToFace:
        if dictOfMaxedVertDistanceToFace[vertex] < closestVertDist:
            closestVert = vertex
            closestVertDist = dictOfMaxedVertDistanceToFace[vertex]
    return(closestVert)

# New Tiebreaker

In [11]:
## For each maxed vertex, find distance to each covered faces
## Can refernce to vertexToVertex for distances I think
## Find minimum dist to touched face, and count how mnay faces are that distance
## Can have 2d array that holds info
    ## 3 Columns, vertex, minDist, and countOfMinDist

def findNextVertexFixed(listOfMaxedFaceVerts):
    
    ## Fill out all vert touching a covered face
    verticesTouchingCoveredFaces = []
    for coveredFace in masterCoveredFaces:
        for vertex in coveredFace:
            if vertex not in verticesTouchingCoveredFaces:
                #print( vertex, "isn't in verticesTouchingCoveredFaces")
                verticesTouchingCoveredFaces.append(vertex)
                #print("verticesTouchingCoveredFaces is now ", verticesTouchingCoveredFaces)
                
    ##for each maxUntouched, find distance to each verticesTouchingCoveredFaces
    dictOfMaxedVertDistanceToCoveredFaces = {}            
    for vertex in listOfMaxedFaceVerts:
        dictOfMaxedVertDistanceToCoveredFaces[vertex] = []
        for vertOnFace in verticesTouchingCoveredFaces:
            dictOfMaxedVertDistanceToCoveredFaces[vertex] = np.append(dictOfMaxedVertDistanceToCoveredFaces[vertex], vertexToVertexDistance[vertex-1][vertOnFace -1])
            dictOfMaxedVertDistanceToCoveredFaces[vertex] = dictOfMaxedVertDistanceToCoveredFaces[vertex].astype(int)
    #print("")
    #print("dictOfMaxedVertDistanceToCoveredFaecs is now:")
    #print(dictOfMaxedVertDistanceToCoveredFaces)
    #print("")
    
    ##Find total minimum distance to a covered face
    minDist = 999999
    for x in dictOfMaxedVertDistanceToCoveredFaces:
        if min(dictOfMaxedVertDistanceToCoveredFaces[x]) < minDist:
            minDist = min(dictOfMaxedVertDistanceToCoveredFaces[x])
            
    #print("")
    #print("min distance is", minDist)
    #print("")
    
    
    ##For each maxUntouched find how many faces are the minimum distances away
    ##Store # of faces closest to maxedVerts
    dictOfNumberClosestFaces = {}
    for vertex in dictOfMaxedVertDistanceToCoveredFaces:
        dictOfNumberClosestFaces[vertex] = []
        count = (dictOfMaxedVertDistanceToCoveredFaces[vertex] == minDist).sum()
        dictOfNumberClosestFaces[vertex] = count
    #print("Lookie here" , dictOfNumberClosestFaces)
    maxedClosestFaces = max(dictOfNumberClosestFaces.values())
        
        ## Stores all maxed
    vertsToChooseFrom = []
    for vertex in dictOfNumberClosestFaces:
        if dictOfNumberClosestFaces.get(vertex) == maxedClosestFaces:
            vertsToChooseFrom = np.append(vertsToChooseFrom, vertex)
    vertsToChooseFrom = vertsToChooseFrom.astype(int)
    ##Pick one randomly / take smallest or largest
    closestVert = random.choice(vertsToChooseFrom)
    #closestVert = vertsToChooseFrom[0]
        
    return(closestVert)

# Randomly select starting vertex

In [12]:
## Select Random Starting vertex
startingVertex = random.randint(25,maxVert+1)
lastVisitedVertex = copy.deepcopy(startingVertex)
solutionVertices = list()


In [13]:
import time

tic = time.time()

print("We are starting at " , startingVertex)
continueAddingVertices = True

while continueAddingVertices == True:
    
    
    ## Add vertex to a solution attempt
    solutionVertices.append(lastVisitedVertex)
    #print(solutionVertices)
    
    
    # Processes Faces Touched by current vertex
    # Counts faces as covered for master array
    # Removes face from each vertex's untouched faces dictionary
    
    ## Find touched faces
    currentTouchedFaces = vertexDictOfTouchedFaces[lastVisitedVertex]
    #print(currentTouchedFaces)


    ## Count all as "covered"
        ## Add to a set of "covered faces"
    ##Needed to set size of array, make sure to remove later!!!!!!!!!!!!!!!!!!!!!!!!
    facesToBeAdded = np.ndarray(shape = (1, len(inputMatrix[0])))
    facesToBeAdded.fill(-1)
    facesToBeAdded = facesToBeAdded.astype(int)
    for x in currentTouchedFaces:
        facesToBeAdded = np.vstack([facesToBeAdded, x])

    #print(facesToBeAdded)

    masterCoveredFaces = np.vstack([masterCoveredFaces, facesToBeAdded])

    ##Removes the size setting rows
    removed = (masterCoveredFaces == -1).sum(1)
    fixed = masterCoveredFaces[removed == 0, :]


    masterCoveredFaces = copy.deepcopy(fixed)

    #print(masterCoveredFaces)



    ## Remove from the current vertex's list of uncovered faces
    for x in vertexDictOfTouchingUncoveredFaces:
        ##make temp
        tempXTouchingUncovered = np.ndarray(shape = (1, len(inputMatrix[0])))
        tempXTouchingUncovered.fill(-1)
        tempXTouchingUncovered = tempXTouchingUncovered.astype(int)
        for y in vertexDictOfTouchingUncoveredFaces[x]:
            ## make inZ
            inZ = False
            for z in vertexDictOfTouchedFaces[lastVisitedVertex]:
                if np.array_equal(y,z):
                    #Set inZ to true
                    inZ = True
                    #print("These are equal")
            ## if not inZ, add to temp
            if not inZ:
                tempXTouchingUncovered = np.vstack([tempXTouchingUncovered, y])
        ## fix temp
            #print(tempXTouchingUncovered)
            removed = (tempXTouchingUncovered == -1).sum(1)
            fixed = tempXTouchingUncovered[removed == 0, :]
            #print("Temp list ", x, fixed)
            tempXTouchingUncovered = copy.deepcopy(fixed)
        
        
        removed = (tempXTouchingUncovered == -1).sum(1)
        fixed = tempXTouchingUncovered[removed == 0, :]
        #print(x, fixed)
        tempXTouchingUncovered = copy.deepcopy(fixed)

        ## make temp x's uncovered touching
        #print("Our temp stuff is: ", x,tempXTouchingUncovered)
        vertexDictOfTouchingUncoveredFaces[x] = copy.deepcopy(tempXTouchingUncovered)
        #print("this",vertexDictOfTouchingUncoveredFaces[x])



    #print(vertexDictOfTouchingUncoveredFaces[lastVisitedVertex])
    
    
    
    
    
    
    # Removes faces touched by current vertex from master uncovered array
    ## Remove touched faces from "uncovered" list of faces
    ## Removing is hard when not sure which row location removing from.
    ## Maybe try just rewriting without the covered ones?

    #tempUncoveredFaces = array([[-1,-1,-1,-1]])
    tempUncoveredFaces = np.ndarray(shape = (1, len(inputMatrix[0])))
    tempUncoveredFaces.fill(-1)
    tempUncoveredFaces = tempUncoveredFaces.astype(int)
    #print(vertexDictOfTouchingUncoveredFaces)

    for x in masterUncoveredFaces:
        inY = False
        for y in vertexDictOfTouchedFaces[lastVisitedVertex]:
            #print(vertexDictOfTouchedFaces[lastVisitedVertex])
            if np.array_equal(x,y):
                inY = True
        if not inY:
            tempUncoveredFaces = np.vstack([tempUncoveredFaces,x])
    #print(tempUncoveredFaces)

    removed = (tempUncoveredFaces == -1).sum(1)
    fixed = tempUncoveredFaces[removed == 0, :]

    masterUncoveredFaces = copy.deepcopy(fixed)

    #print("Faces Not Yet Covered") 
    #print(masterUncoveredFaces)
    #print("")
    #print("Faces covered") 
    #print(masterCoveredFaces)
    #print("")
    
      ## If yes, output list of vertices as solution
    ## Covered face must be 1 version of all, and uncovered must be empty
    if len(masterUncoveredFaces) == 0:
        print("Solution Set")
        print(solutionVertices)
        print(len(solutionVertices), " vertices in solution set")
        #print( " ")
        #print("Uncoverd faces")
        #print(masterUncoveredFaces)
        #print( " ")
        #print("Covered Faces")
        #print(masterCoveredFaces)
        continueAddingVertices = False
        break
    
    
    #print(lastVisitedVertex)
    ## If no, select next Vertex
        
        
    # Find all vertices with highest uncovered faces
    
    ## Finding Next Vertex
    ## Find all vertices that touch maximum number of uncovered faces
    
    #print(vertexDictOfTouchingUncoveredFaces)
    
    maxUntouched = 0
    maxUntouchedVerts = []
    for x in vertexDictOfTouchingUncoveredFaces:
        if len(vertexDictOfTouchingUncoveredFaces[x]) > maxUntouched:
            maxUntouched = len(vertexDictOfTouchingUncoveredFaces[x])
        #print(x, len(vertexDictOfTouchingUncoveredFaces[x]))
        #print("max ", maxUntouched )

    for x in vertexDictOfTouchingUncoveredFaces:
        if len(vertexDictOfTouchingUncoveredFaces[x]) == maxUntouched:
                maxUntouchedVerts.append(x)
    print("Vertices with maximum untouched faces, [", maxUntouched, "] are: ",maxUntouchedVerts)
    #print(maxUntouchedVerts[0])


        ## Calculate which of vertices in previous step are closest to a covered face 
    if len(maxUntouchedVerts) == 1:
        lastVisitedVertex = maxUntouchedVerts[0]
    elif len(maxUntouchedVerts) == 0:
        print("SOMETHING IS VERY WRONG HERE, NO MAXIMIZED VERTS")
    else:
        lastVisitedVertex = findNextVertexFixed(maxUntouchedVerts)
        print("We are now going to ",lastVisitedVertex)


    #print("The next vertex is" , lastVisitedVertex)

    
    # Check if all faces covered, if yes, print out solution, covered and uncovered
    ## Check if all faces covered
    #print(inputMatrix)
    #print(masterCoveredFaces)
    #print(np.array_equal(inputMatrix, masterCoveredFaces))

toc = time.time()

print(toc - tic)
  


We are starting at  86
Vertices with maximum untouched faces, [ 20 ] are:  [1, 2, 3, 4, 5, 6, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 36, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 82, 83, 85, 87, 88, 89, 90, 91, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120]
We are now going to  82
Vertices with maximum untouched faces, [ 20 ] are:  [1, 2, 3, 4, 5, 6, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 26, 27, 29, 31, 33, 34, 36, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 85, 87, 89, 90, 91, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120]
We a

We are now going to  4
Vertices with maximum untouched faces, [ 2 ] are:  [17, 40, 89, 99]
We are now going to  99
Vertices with maximum untouched faces, [ 2 ] are:  [40, 89]
We are now going to  89
Vertices with maximum untouched faces, [ 2 ] are:  [40]
Vertices with maximum untouched faces, [ 1 ] are:  [6, 14, 17, 23, 34, 35, 38, 53, 54, 55, 58, 60, 61, 66, 74, 76, 87, 98, 104, 117]
We are now going to  55
Vertices with maximum untouched faces, [ 1 ] are:  [6, 14, 17, 34, 35, 38, 53, 54, 60, 61, 66, 74, 76, 87, 98, 117]
We are now going to  54
Vertices with maximum untouched faces, [ 1 ] are:  [14, 17, 34, 35, 38, 53, 60, 61, 74, 87, 98, 117]
We are now going to  98
Vertices with maximum untouched faces, [ 1 ] are:  [14, 34, 35, 38, 53, 60, 61, 87]
We are now going to  87
Vertices with maximum untouched faces, [ 1 ] are:  [34, 53, 60, 61]
We are now going to  60
Solution Set
[86, 82, 85, 39, 9, 27, 1, 10, 93, 63, 41, 62, 71, 114, 107, 100, 101, 59, 119, 103, 47, 64, 96, 95, 8, 65, 72

In [14]:
fullSolution = set(list(range(1,25))).union(set(solutionVertices))
print(fullSolution)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 36, 39, 40, 41, 47, 51, 54, 55, 56, 59, 60, 62, 63, 64, 65, 70, 71, 72, 73, 75, 82, 83, 85, 86, 87, 89, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 106, 107, 109, 110, 112, 113, 114, 119}


<ipython-input-12-956e056d23c1> in <module>
    172         print("SOMETHING IS VERY WRONG HERE, NO MAXIMIZED VERTS")
    173     else:
--> 174         lastVisitedVertex = findNextVertexFixed(maxUntouchedVerts)
    175         print("We are now going to ",lastVisitedVertex)
    176 

<ipython-input-10-07da577904fc> in findNextVertexFixed(listOfMaxedFaceVerts)
     31     minDist = 999999
     32     for x in dictOfMaxedVertDistanceToCoveredFaces:
---> 33         if min(dictOfMaxedVertDistanceToCoveredFaces[x]) < minDist:
     34             minDist = min(dictOfMaxedVertDistanceToCoveredFaces[x])
     35 

ValueError: min() arg is an empty sequence

In [15]:
solutionConnections = {}

for i in fullSolution:
    solutionConnections[i] = []
    for j in vertexDictOfSharedEdgesEnding[i]:
        
        if j in fullSolution:
            solutionConnections[i].append(j)
print(solutionConnections)

for i in solutionConnections:
    print(i, len(solutionConnections[i]))

{1: [26, 36, 40], 2: [62, 27, 36, 39, 56, 64], 3: [73, 40, 41, 51, 47, 70], 4: [62, 36, 41, 60, 63], 5: [73, 26, 27, 47, 54, 56, 75], 6: [62, 73, 56, 59, 60, 71, 54, 65], 7: [86, 97, 83, 95, 89, 100], 8: [86, 93, 87], 9: [97, 26, 99], 10: [97, 40, 94, 51, 98], 11: [86, 39, 82], 12: [87, 113, 85, 107, 93, 95, 89], 13: [113, 41, 70, 93, 94, 114], 14: [63, 87, 39, 109, 85, 64], 15: [63, 113, 60, 110, 107, 71, 65], 16: [98, 114, 94, 95, 96, 112, 100], 17: [98, 51, 47, 72, 96, 75, 99], 18: [114, 70, 71, 72, 119, 112], 19: [99, 103, 83, 96, 102, 100, 106], 20: [103, 27, 82, 55], 21: [75, 103, 59, 54, 72, 119, 55, 102], 22: [82, 83, 101, 109, 85, 89, 106], 23: [64, 59, 109, 110, 55, 101, 65], 24: [101, 102, 110, 107, 112, 119, 106], 26: [1, 5, 27, 9], 27: [2, 56, 5, 26, 20], 36: [1, 2, 39, 62, 4], 39: [2, 36, 64, 11, 14], 40: [1, 3, 41, 51, 10], 41: [3, 40, 70, 4, 13], 47: [3, 73, 51, 5, 54, 75, 17, 72], 51: [3, 40, 47, 10, 98, 17], 54: [5, 47, 56, 73, 75, 6, 59, 21, 72], 55: [20, 103, 21, 10